<a href="https://colab.research.google.com/github/YourUsername/YourRepo/blob/main/recover_grade_colab_debug.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recovering Alice's Grade (with Debugging)

This notebook attempts to recover Alice's grade from an encrypted ciphertext using her RSA public key. The grade is assumed to be a small integer (0–100), and the encryption is done without padding, allowing a brute-force approach. Debugging output is included to diagnose issues.

## Prerequisites
- The `cryptography` library is required.
- The public key is provided in PEM format.
- The ciphertext is provided in hexadecimal format.

In [ ]:
!pip install cryptography

In [ ]:
import base64
import binascii
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import serialization

# Alice's public key in PEM format
public_key_pem = """-----BEGIN PUBLIC KEY-----
MIGfMA0GCSqGSIb3DQEBAQUAA4GNADCBiQKBgQDKClTqiJTUa++IPogThEsiNR4J
FpmV12jbfYvEc74ZtyCxGYpt3UcwaYbBoVgBpFepBRnwjJEPX8jxip7yfxr/vqYv
MrQ4LJggKRKUDrWFwuI+lNmxsVz+E4now0v1E/lHa5p8PxdqRBdm1xw4yXx48Xft
rnnCa8w19lq20OSNPwIDAQAB
-----END PUBLIC KEY-----"""

# Ciphertext (hex representation of encrypted grade)
ciphertext_hex = "9A60E4CE8D70B2A12BB2422D73571A445159955A844AE5EA9995870AA4819BA434835C88AB4F1FBD17712DC525613382FF6A9621CB9BC0F82191EB60AAA369FC061A614C18F81FA9906FB168E0E8B0A0EA5C3A9E6E1566820E4831CAA9BDF0FB048F8095DE65DB6D9FA79AFF7D40529E512ADB91231D176944064200AEC070A1"

try:
    # Load the public key
    public_key = serialization.load_pem_public_key(public_key_pem.encode())
    print("Public key loaded successfully.")

    # Extract modulus and exponent for debugging
    public_numbers = public_key.public_numbers()
    print(f"Public key modulus (n): {public_numbers.n}")
    print(f"Public key exponent (e): {public_numbers.e}")

    # Convert hex ciphertext to bytes
    ciphertext = binascii.unhexlify(ciphertext_hex)
    print(f"Ciphertext length: {len(ciphertext)} bytes")
    print(f"Ciphertext (first 10 bytes): {ciphertext[:10].hex()}")

    # Function to encrypt a number using the public key (no padding)
    def encrypt_no_padding(number, public_key):
        # Convert number to bytes (big-endian)
        number_bytes = number.to_bytes((number.bit_length() + 7) // 8 or 1, byteorder='big')
        print(f"Trying grade {number}, plaintext bytes: {number_bytes.hex()}")
        # Perform raw RSA encryption: c = m^e mod n
        encrypted = public_key.encrypt(
            number_bytes,
            padding=None
        )
        return encrypted

    # Brute-force possible grades (0 to 100)
    found = False
    for grade in range(101):
        try:
            # Encrypt the grade
            encrypted_grade = encrypt_no_padding(grade, public_key)
            print(f"Encrypted grade {grade} length: {len(encrypted_grade)} bytes")
            print(f"Encrypted grade {grade} (first 10 bytes): {encrypted_grade[:10].hex()}")
            # Compare with the given ciphertext
            if encrypted_grade == ciphertext:
                print(f"Found Alice's grade: {grade}")
                found = True
                break
        except Exception as e:
            print(f"Error encrypting grade {grade}: {str(e)}")
            continue

    if not found:
        print("No matching grade found in range 0–100.")

    # Additional test for grade 87 specifically
    print("\nTesting grade 87 explicitly:")
    try:
        encrypted_87 = encrypt_no_padding(87, public_key)
        print(f"Encrypted grade 87 (first 10 bytes): {encrypted_87[:10].hex()}")
        print(f"Target ciphertext (first 10 bytes): {ciphertext[:10].hex()}")
        if encrypted_87 == ciphertext:
            print("Confirmed: Grade 87 matches the ciphertext.")
        else:
            print("Grade 87 does not match the ciphertext.")
    except Exception as e:
        print(f"Error encrypting grade 87: {str(e)}")

except Exception as e:
    print(f"Error in setup: {str(e)}")

## How It Works

- **Public Key**: Loads Alice’s RSA public key from PEM format and prints the modulus and exponent for verification.
- **Ciphertext**: Converts the hex ciphertext to bytes and displays its length and first 10 bytes for debugging.
- **Brute-Force**: Tries grades 0–100, encrypting each with the public key without padding. For each grade, it prints the plaintext bytes and the encrypted result’s length and first 10 bytes.
- **No Padding**: Uses `padding=None` for raw RSA encryption (c = m^e mod n), matching the problem’s context.
- **Debugging**: Includes error handling and logs to identify issues. Tests grade 87 explicitly to confirm the expected result.

## Troubleshooting
If no match is found:
- Check the debugging output for errors during encryption.
- Compare the ciphertext length with the encrypted grade lengths (should match, typically 128 bytes for a 1024-bit key).
- Verify the `cryptography` library version (`!pip show cryptography`).
- Ensure the ciphertext hex is exactly as provided.
- If the plaintext was encoded differently (e.g., as a string '87'), modify the `encrypt_no_padding` function to test alternative encodings.